# Iris de fisher

https://fr.wikipedia.org/wiki/Iris_de_Fisher

## chargement des données

https://seaborn.pydata.org/generated/seaborn.load_dataset.html

On charge les données da,s ine variable nommée iris.

> **NB:** dans un notebook la dernière ligne est par défaut affichée à l'écran (si ce n'est pas une affectation). On prend donc l'habitude, aprèsune affectation de remettre le nom de l'objet affecté.

In [2]:
import seaborn as sns

In [3]:
iris = sns.load_dataset('iris')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
iris.dtypes

5 colonnes, donc 4 réelles

In [ ]:
iris.shape

150 lignes et 5 colonnes

C'est comparable aux données cvs de https://github.com/mwaskom/seaborn-data/blob/master/iris.csv

## Statistiques descriptives

### Statistiques globales

In [ ]:
iris.describe()

Seuls les descriptionsdes caractères numériques sont représentées (donc pas 'species' ici)

Pour séparer les espèces, on utilise https://wiki.centrale-marseille.fr/informatique/public:appro-s7:td6

Pour les 'setosa' :

In [ ]:
iris[iris['species'] == 'setosa'].describe()

En plus funky, sans connaissances a priori sur les données.  On commence par trouver toutes les valeurs différentes de la colonne species :

In [ ]:
iris['species'].unique()

On peut ensuite faire une boucle :

In [ ]:
for name in iris['species'].unique():
    print("species:", name)
    display(iris[iris['species'] == name].describe())

### représentation graphique

On va représenter un boxplot (https://fr.wikipedia.org/wiki/Bo%C3%AEte_%C3%A0_moustaches) de nos données.

Pour représenter graphiquement les choses, on utilise matplotlib. On va toujours procéder de la même manière : 

1. créer le graphique avec matplotlib
2. ajouter des choses au dessin
3. représenter la figure

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 1. créer le dessin (ici ax)
fig, ax = plt.subplots(figsize=(7, 7)) 

#  2. ajouter des choses au dessin
sns.boxplot(data=iris,   
            ax=ax)

# 3. représenter le graphique
plt.show() 

boxblot par espèces, pour un caractère.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

sns.boxplot(x='species',
            y= 'petal_length',
            data=iris,              
            ax=ax)

plt.show() 

On utilise les subplot de matplotlib pour faire autant de graphiques que de colonnes. On crée ci-dessous 2 lignes de 2 graphiques horizontaux (les 2 premiers paramètres), on représente ensuite chaque boxplot dans son dessin.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 20)) 

sns.boxplot(x='species',
            y= 'petal_length',
            data=iris,              
            ax=ax[0,0])

sns.boxplot(x='species',
            y= 'sepal_length',
            data=iris,              
            ax=ax[1,0])

sns.boxplot(x='species',
            y= 'petal_width',
            data=iris,              
            ax=ax[0,1])

sns.boxplot(x='species',
            y= 'sepal_width',
            data=iris,              
            ax=ax[1,1])


plt.show() 

Les caractères deux à deux, avec une séparation par espèce. Voir https://seaborn.pydata.org/generated/seaborn.pairplot.html?highlight=pairplot#seaborn-pairplot pour la documentation. La méthode pairplot ne possède pas de paramètre *ax*. Il crée lui-même la figure. Si on veut ajouter des éléments à la figure, il faut récuperer le dessin créé par pairplot.

In [ ]:
sns.pairplot(iris, hue="species", markers=["o", "s", "D"])
plt.show()

## corrélations

### globale

Les setosa ont l'air d'être les *"petites"* iris. En regardant les courbes, on a l'impresion que la largeur du petale est corrélée positivement à la longueur du petal et la longuer du sépale. En revanche ce n'est pas le cas de la largeur du sépale (grande et petite largeur de sépalle pour une largeur de petal donné.

In [ ]:
iris.corr()

En regardant la ligne (ou colonne, la matrice est symétrique) dédiée à la largeur du pétale on voit bien que la longueur du petale et la longueur du sépale sont corrélés très positivement (0.82 et 0.96 respectivement) alors que la largeur du sépale est proche de 0 (-.37).

Les caractères les moins corrélées linéairements sont longueur du sépale et largeur du sépale (-.12). Vérifions le graphiquement. En traçant la régression entre ces deux caractères. On utilise la méthdoe https://seaborn.pydata.org/generated/seaborn.regplot.html?highlight=regplot#seaborn.regplot de seaborn

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
sns.regplot(x=iris['sepal_length'],
            y=iris['sepal_width'],
            ci=False,
            ax=ax
          )
plt.show()

On calcule la droite de régression avec statmodels. 

**tuto :** https://statsmaths.github.io/stat289-f18/solutions/tutorial15-statsmodels.html

On va essayer d'expliquer la largeur du sépale par la longueur du sépale : $largeur = a * longueur + b$. Par défaut, statsmodels ne calcule pas le paramètre *b*, il cherche une régression passant par 0 : $largeur = a * longueur$

Il faut donc lui demander de rajouter cette constante avec la méthode *statsmodels.api.add_constant*. 

Statsmodels fonctionne en 2 temps : 
1. on choisit le modèle de régression, ici : $largeur = a * longueur + b$
2. on calcule la régression (avec la méthode *fit*)

In [ ]:
import statsmodels.api

model = statsmodels.api.OLS(iris['sepal_width'], statsmodels.api.add_constant(iris['sepal_length']))
regression = model.fit()
print(regression.summary())

Le coafficient de corrélation $R^2$ (valeur *R-squared*) vaut 0.014 (donc $|R|$ vaut bien 0.12), la constantante $b$ vaut 3.41 (*const*)  et le coefficient directeur $a$ -0.061 (*sapal_length*).

Utilisons *sns.pairplot* pour tracer toutes les droites de corrélations en même temps.

**ATTENTION** : veririfer toujours la corrélation avant de tracer les droites de régression. Une droite de régression avec un petti coefficient de corrélation n'est pas pertinente.

In [ ]:
sns.pairplot(iris, kind="reg")
plt.show()

Pour faire des dessins multiples avec seaborn, vous pouver lire le tuto suivant : https://seaborn.pydata.org/tutorial/axis_grids.html#building-structured-multi-plot-grids

### Par classes

In [ ]:
iris.groupby('species').corr()

sous la forme d'une *heatmap* :

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(iris.groupby('species').corr(), annot=True)
plt.show()

Intra espèce, largeur et longuer de sépale sont bien plus corrélés.

In [ ]:
sns.pairplot(iris, hue="species", kind="reg")
plt.show()

## Régression logisitique

Les *virginica* ont l'air d'être les grandes iris. Faisons une régression logistique pour voir à partir de quand décider si une espèce est *virginica* ou pas.

### pour un caractère

On commence par créer une colonne qui faut 1 si l'iris est sétosa, et 0 sinon. La fonction ajoute prend un DataFrame en paramètre et rend une liste qui sera notre nouvelle colonne. 

Cette fonction est ensuite passée en paramètre de la méthode *assign* qui créera la nouvelle colonne (nommée 'est_virginica')

In [6]:
import numpy as np

iris_logistique = iris.assign(est_virginica=np.where(iris['species'] == 'virginica', 1, 0))

iris_logistique

,sepal_length,sepal_width,petal_length,petal_width,species,est_virginica
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,1
146,6.3,2.5,5.0,1.9,virginica,1
147,6.5,3.0,5.2,2.0,virginica,1
148,6.2,3.4,5.4,2.3,virginica,1


On peut maintenant supprimer la colonne species de notre dataframe

In [7]:
iris_logistique = iris_logistique.drop('species', axis='columns')

iris_logistique

,sepal_length,sepal_width,petal_length,petal_width,est_virginica
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


La régression logistique va essayer de savoir à partir de quelles corrélation linéaire de caractères on peut dire qu'une iris est setosa ou pas.

**définition** : https://en.wikipedia.org/wiki/Logistic_regression


Le modèle est :

$$ln(\frac{\mbox{probabilité d'être virginica}}{\mbox{probabilité de ne pas être virginica}}) = a * petal\_length + b$$

Ce qui  donne une mesure de probabilité en forme d'une courbe logistique. Une fois les paramètre trouvé on aura que :
* la probabilité d'être virginica < 1/2 si : $0 > a * petal\_length + b$
* la probabilité d'être virginica = 1/2 si : $0 = a * petal\_length + b$
* la probabilité d'être virginica > 1/2 si : $0 < a * petal\_length + b$

La pente étant d'autant plus raide selon la valeurs des $a$.

Si on jargonne, On a une droite séparant les probabilités inférieures et supérieures à 1/2.

Pour statsmodels on doit avoir :
- la colonne à expliquer (la colonne 'est_virginica')
- la (ou les) colonne(s) qui expliquent (la colonne 'petal_length')
- on ajoute la constante

**NB** : Si on avait fait la même chose avec les 'setosa' on aurai eu une erreur de type `PerfectSeparationError`. Les setosa sont parfaitement séparés, la pente de la courbe logistique est verticale, d'où une erreur.

Essayons une régression logisitique que sur la la longueur du pétale.

In [8]:
import statsmodels.discrete.discrete_model
import statsmodels.api

log_reg = (statsmodels.discrete.discrete_model
            .Logit(iris_logistique['est_virginica'], 
                   statsmodels.api.add_constant(iris_logistique['petal_length']))
            .fit())

log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.111440
         Iterations 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          est_virginica   No. Observations:                  150
Model:                          Logit   Df Residuals:                      148
Method:                           MLE   Df Model:                            1
Date:                Wed, 11 Jan 2023   Pseudo R-squ.:                  0.8249
Time:                        17:57:55   Log-Likelihood:                -16.716
converged:                       True   LL-Null:                       -95.477
Covariance Type:            nonrobust   LLR p-value:                 3.936e-36
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -43.7809     11.110     -3.941      0.000     -65.556     -22.006
petal_length     9.0020      2.283      3.943      0.000       4.528      13.476
================================================================================

Possibly complete quasi-separation: A fraction 0.47 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

La probabilité de 1/2 est pour une longeur de pétal de 43.7809/9.0020 = 4.863463674738947

Dessinons le tout avec seaborn :

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.regplot(x=iris_logistique['petal_length'],
            y=iris_logistique['est_virginica'],
            ci=False,
            logistic=True,
            ax=ax
          )
plt.show()

### pour tous les caractères

Le modèle est :

$$ln(\frac{\mbox{probabilité d'être virginica}}{\mbox{probabilité de ne pas être virginica}}) = a_0 * sepal\_length + a_1 * sepal\_width + a_2 * petal\_length * a_3 * petal\_width + b$$

Ce qui  donne une mesure de probabilité en forme d'une courbe logistique. Une fois les paramètre trouvé on aura que :
* la probabilité d'être virginica < 1/2 si : $0 > a_0 * sepal\_length + a_1 * sepal\_width + a_2 * petal\_length * a_3 * petal\_width + b$
* la probabilité d'être virginica = 1/2 si : $0 = a_0 * sepal\_length + a_1 * sepal\_width + a_2 * petal\_length * a_3 * petal\_width + b$
* la probabilité d'être virginica > 1/2 si : $0 < a_0 * sepal\_length + a_1 * sepal\_width + a_2 * petal\_length * a_3 * petal\_width + b$

La pente étant d'autant plus raide selon la valeurs des $a_i$.

Si on jargonne, On a un hyerplan séparant les probabilités inférieures et supérieures à 1/2.

Pour statsmodels on doit avoir :
- la colonne à expliquer (la colonne 'est_virginica')
- les colonnes qui expliquent (les 4 paramètres. C'est à dire la dataframe sans la colonne 'est_virginica')
- on ajoute la constante

In [ ]:
import statsmodels.discrete.discrete_model
import statsmodels.api

log_reg = (statsmodels.discrete.discrete_model
            .Logit(iris_logistique['est_virginica'], 
                   statsmodels.api.add_constant(iris_logistique.drop('est_virginica', axis='columns')))
            .fit())

log_reg.summary()

L'effet des paramètre sur la probabilité peut être trouvé avec les dérivées :

In [ ]:
log_reg.get_margeff(at='overall', method='dydx', count=True).summary()

La longeur et la largeur du petal à une action positive sur le fait d'être une virginica, alors que le sepal n'a pas l'air de jouer beaucoup.